In [ ]:
from IPython.display import display, HTML, clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import prodimopy.read as pread
import pickle
import numpy as np
import pandas as pd
import time
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

In [96]:
import glob

wlrange = [5, 28]
species = ['C2H2_H', 'CH4_H', 'CO', 'CO2_H', 'HCN_H', 'NH3_H', 'NO_H', 'OH', 'S', 'o-H2', 'o-H2O', 'p-H2', 'p-H2O']
ions = ['Ar+', 'Ar++', 'Fe+', 'Ne+', 'Ne++']
values = np.arange(-0.5, 0.51, 0.25)
data = []
start_time = time.time()
i = 1
for C in values:
    for O in values:
        start_time = time.time()
        
        model_data = []
        
        path = 'models/C_' + str(C) + '_O_' + str(O) + '_CO_*'
        matching_file = glob.glob(path)
        
        CO = matching_file[0].split('CO_')[-1]
        
        model_data.append(matching_file[0][7:])
        model_data.append(str(C))
        model_data.append(str(O))
        model_data.append(str(CO))

        model = pread.read_prodimo(matching_file[0], filenameFLiTs='specFLiTs1.out')

        wl, flux, flux_cont = model.FLiTsSpec.convolve(3000, contReturn=True)
        flux -= flux_cont
        model_data.append(wl)
        model_data.append(flux)

        for specie in species:
            model_data.append(model.gen_specFromLineEstimates(ident=specie, wlrange=wlrange, noCont=True, unit='Jy')[1])
        
        total_flux_ion = None
        for ion in ions:
            wl, flux_ion = model.gen_specFromLineEstimates(ident=ion, wlrange=wlrange, noCont=True, unit='Jy')
            if total_flux_ion is None:
                total_flux_ion = flux_ion
            else:
                total_flux_ion += flux_ion
        model_data.append(total_flux_ion)
        model_data.append(wl)
        
        data.append(model_data)
        
        end_time = time.time()
        clear_output()
        current_time = end_time - start_time
        print(f'{i}/25; {int(current_time // 60)}:{int(current_time % 60)}')
        i += 1


25/25; 2.0:26.637526750564575


In [97]:
df = pd.DataFrame(data,
                  columns=['Model Name', 'C Value', 'O Value', 'CO Value', 'FLiTs Wavelength', 'FLiTs Flux', 'C2H2_H',
                           'CH4_H', 'CO', 'CO2_H', 'HCN_H', 'NH3_H', 'NO_H', 'OH', 'S', 'o-H2', 'o-H2O', 'p-H2',
                           'p-H2O', 'Ion', 'Prodimo Wavelength'])

with open('ModelData.pkl', 'wb') as f:
    pickle.dump(df, f)
